# Packages and Plotting

## Topics
- Packages
- Plotting & visualization

## Packages
Julia has over 4000 registered packages, making packages a huge part of the Julia ecosystem.

Even so, the package ecosystem still has some growing to do. Notably, we have first class function calls to other languages, providing excellent foreign function interfaces. We can easily call into python or R, for example, with `PyCall` or `Rcall`.

This means that you don't have to wait until the Julia ecosystem is fully mature, and that moving to Julia doesn't mean you have to give up your favorite package/library from another language!

To see all available packages, check out

https://github.com/JuliaRegistries/General or https://juliaobserver.com/

## Installing packages
For now, let's learn how to use a package.

Before anything, we have to activate the package manager by running `using Pkg`.

The first time you use a package on a given Julia installation, you need to explicitly add it

In [ ]:
using Pkg
Pkg.add("Example")

## Using packages
Every time you use Julia (start a new session at the REPL, or open a notebook for the first time, for example), you load the package with the `using` keyword

In [ ]:
using Example

In the source code of `Example.jl` at https://github.com/JuliaLang/Example.jl/blob/master/src/Example.jl we see the following function declared

```julia
hello(who::String) = "Hello, $who"
```
Having loaded `Example`, we should now be able to call `hello`

In [ ]:
hello("Julia!")

## Import

In addition to the `using` keyword, packages can be loaded with the `import` keyword.

In [ ]:
import Example

Import does not add any functions to the main name space, so you cannot call them directly, as above. They can only be accessed using a dot-syntax:

In [ ]:
Example.hello("Julia!")

Import can be aliased to make referring to them easier:

In [ ]:
import Example as e
e.hello("Julia!")

It also allows you to pull a single name from the package:

In [ ]:
import Example.hello as hello2
hello2("Julia!")

### Notebook `09_ecosystem` has a list of packages and additional information.

- Search for packagers on the [Julia Hub](https://juliahub.com/)
- The [package registry](https://github.com/JuliaRegistries/General)


### Exercise 1: Installing a package: `Primes`

Load the `Primes` packages (source code at https://github.com/JuliaMath/Primes.jl). 

Verify that you can now use the function `primes` to grab all the primes under `100`. If unsure, maybe `?primes` will help you?

In [ ]:
using Pkg
# use Pkg to "add" the package "Primes"
...


In [ ]:
# load the Primes package
...

In [ ]:
# How do you list the first 100 primes?


## Interoperability
There are many existing codes and libraries, why should we re-use them? One of the strenghts of Julia is it's interoperability. 

C and Fortran, for example are natively supported. But many other languages are just one package away. 

Take python for example:

In [ ]:
using Pkg
Pkg.add("PyCall")
using PyCall

@pyimport scipy.optimize as so
so.newton(x -> cos(x) - x, 1)

Or calling a function from the C standard library:

In [ ]:
t = ccall(:clock, Int32, ())

In [ ]:
# Or specifying the library (necessary if it's not standard)
ccall((:printf, "libc.so.6"), Int32, (Cstring,), "Hello!")

## Different libraries for different languages
- Python with [PyCall](https://github.com/JuliaPy/PyCall.jl)
- R with [RCall](https://github.com/JuliaInterop/RCall.jl)
- C++ with [CxxWrap](https://github.com/JuliaInterop/CxxWrap.jl)
- Matlab with [MATLAB](https://github.com/JuliaInterop/MATLAB.jl)
- Java with [JavaCall](https://github.com/JuliaInterop/JavaCall.jl)
- Mathematica with [Mathematica](https://github.com/JuliaInterop/Mathematica.jl)
- ObjectiveC with [ObjectiveC](https://github.com/JuliaInterop/ObjectiveC.jl)

### Exercise 2: C

Use the C `cos` function to find the cos π.

In [ ]:
t = ccall(:cos, Float64, (Float64,), π)

### Exercise 3: Python

Use the Python `math.cos` function to find the cos π.

In [ ]:
@pyimport math as pymath
pymath.cos(π)

## Plotting

There are few different ways to plot in Julia. 

Here we'll use the `Plots` [package](http://docs.juliaplots.org/latest/) that tries to unify many of the underlying visualization engines to have a similar interface.

In [ ]:
using Pkg
Pkg.add("Plots")
using Plots

One of the advantages to `Plots.jl` is that it allows you to seamlessly change backends. For example, if you are familiar with the matplotlib library from Python, then you can keep using that via the `PyPlot` [package](https://github.com/JuliaPy/PyPlot.jl).

Engines/backends include:
- [Plotly](https://plot.ly/julia/): Plotly.jl package (`plotly()` to activate)
    - default for Plots if nothing else is installed
    - interactive plots in browser
    - `PlotlyJS` (activate with `plotlyjs()`) is the local offline version
- [PyPlot](https://github.com/JuliaPy/PyPlot.jl): Matplotlib interface (`pyplot()` to activate)
    - All the bells and whistles of Matplotlib
- [GR](https://github.com/jheinen/GR.jl): GR framework interface (`gr()` to activate)
- [UnicodePlots](https://github.com/Evizero/UnicodePlots.jl): (`unicodeplots()` to activate)
    - Plots inside terminal!
    
Also:
- [Gadfly](http://gadflyjl.org/stable/): Wickham-Wilkinson style grammar
For R-styled plotting.

See the [docs](http://docs.juliaplots.org/v1/backends/) for more info about different backends, and their pros & cons.

In [ ]:
#Pkg.add("PlotlyJS") #let's install our own local plotly
Pkg.add("PyPlot") #let's install our own local plotly

In [ ]:
#plotly()       # Plotly.jl (needs Plotly package)
#plotlyjs()     # local javascript server for plotly (needs PlotlyJS)
#gr()           # GR backend, (needs GR package)
pyplot()       # Matplotlib interface (needs PyPlot package and up-to-date system matplotlib)
#unicodeplots() # UnicodePlots backend (needs UnicodePlots package); NOTE: does not work in notebooks

## Simulation
Let's make a nice plot of the state of our simulation.

In [ ]:
# Since our simulation only has 4 states, we'll make a palette of 4 colors.
palette = distinguishable_colors(4)

Interesting choice. First we will need to assign colors to each plant in the simulation.

In [ ]:
# The following line loads the epidemic functions from a file
include("../epidemic.jl")

# Let's make some plants
plants = make_plants(32,32)

In [ ]:
"Map the plants to colors for plotting"
function to_colors(plant::Plant)
    if plant.status == uninfected
        return RGB(0.0,0.8,0.0)
    end
    if plant.status == infected
        return RGB(0.8,0.0,0.0)
    end
    if plant.status == dead
        return RGB(0.1,0.1,0.1)
    end
    if plant.status == recovered
        return RGB(0.0,0.0,0.8)
    end
    if plant.status == immune
        return RGB(0.2,0.2,1.0)
    end
end

to_colors.(plants)

In [ ]:
# We can pass this to plot() to get a nicer image

plot(to_colors.(plants),legend=false, border=:none)

## Random Walk
Let's take our visualization to the next dimension. Here is a simple plot to visualize the random walk in 3D.

In [ ]:
#activate Pyplot backend (if you have not done it previously)
using Plots
pyplot()

In [ ]:
n = 1000
rw() = cumsum(randn(n)) #function for cumulative random values

# x/y/z coordinates
x = rw() 
y = rw()
z = rw()

plot(x,y,z)

## Animations
Finally, let's add some time aspect to our visualizations. `Plots` has a simple `@gif` macro for this.

It builds an animation using one frame per iteration loop, then creates an animated GIF from that.
```julia
p = plot(1)
@gif for x=0:0.1:5
    push!(p, 1, sin(x))
end
```

Let's write up a simple Lorenz attractor. See https://en.wikipedia.org/wiki/Lorenz_system

In [ ]:
n = 2000
dt = 0.02
σ, ρ, β = 9.9, 28., 8/3
global x, y, z = 1., 1., 1.

# Lorentz attractor
function lorentzAttractor(x,y,z)
    x += dt * σ*(y - x)
    y += dt * (x*(ρ - z) - y)   
    z += dt * (x*y - β*z  )
    return x, y, z
end

In [ ]:
#activate PyPlot backend (if you have not done it previously)
using Plots

In [ ]:
# initialize a 3D plot with 1 empty series
plt = path3d(1, xlim=(-25,25), ylim=(-25,25), zlim=(0,50),
                xlab = "x", ylab = "y", zlab = "z",
                title = "Lorenz Attractor", marker = 1)

# build an animated gif, saving every 10th frame
@gif for i=1:n
    global x, y, z = lorentzAttractor(x,y,z)
    push!(plt, x, y, z)
end every 100

## More examples
For more examples, see [Plots](http://docs.juliaplots.org/latest) documentation.
- [PyPlot](http://docs.juliaplots.org/latest/examples/pyplot/)
- [GR](http://docs.juliaplots.org/latest/examples/gr/)
- [Plotly](http://docs.juliaplots.org/latest/examples/plotlyjs/)
- etc.

### Exercise 4: Simple plots

Given `x = -10:10` plot y vs. x for 
$
y=x^2
$

In [ ]:
using Plots

x = -10:10
y = ...
plot(...)

### Exercise 5: Mount Bruno

Let's visualize Mount Bruno! 

Read the 2D topological data of Mount Bruno from the file `../data/bruno.csv`. Next we need some plotting engine for `Plots`, I recommend `Plotly`/`PlotlyJS` for this task. As a final touch, see what `surface()` function can do with your array.

## Exercise 6: Animating the simulation

Let's create the animation in the first notebook. Epidemic.jl also defines the function `count_infections` and `count_deaths`. Read these and make sure you know what they do.

Below is a skeleton code for constructing the plot. Fill in the missing detail. Feel free to play aroun with the code, but note that there are problems with animating just the left side, so don't be suprised if you see weird color issues.

We use `@animate` instead of `@gif`, because it allows us to save it to a file. 

In [ ]:
# The following two lines load the epidemic functions from a file
include("../epidemic.jl")

In [ ]:
using Plots

# Set the size of the animation window
default(size = (400, 300))

# The map of plants (a 2D array)
plants = make_plants(64, 64)
# List the number of infections for each frame
infections = [count_infections(plants)]
# List the number of deaths for each frame
deaths = [count_deaths(plants)]

""" Build the animation in a function.
    We run this in 
"""
function animation(plants)
    anim = @animate for i ∈ 1:50
        # run update function here
        ...
        
        # append the current number of infections and deaths to the two arrays
        ...
        
        # Create a plot of the simulation (left side in the animation)
        plot1 = plot(...)
        # And a plot of the infection and death counts
        plot2 = plot(...)
        
        # This will set up the layout:
        l = @layout [a b]
        
        # And finally we combine the two plots with the layout
        plot(plot1, plot2, layout = l)
    end
    return anim
end

# Save the animation as a gif
gif(animation(plants), "epidemic.gif", fps = 5)

## Summary

Julia provides a powerful interoparability API for usage of various other languages.

These rely on external libraries such as:
- Python with [PyCall](https://github.com/JuliaPy/PyCall.jl)
- R with [RCall](https://github.com/JuliaInterop/RCall.jl)

Same functionality also allows to use many mature visualization libraries via Julia.